In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from builtins import input

#import system things
from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import os
import keras

Using TensorFlow backend.


In [0]:

class Siamese:

    # Create model
    def __init__(self):
        self.x1 = tf.placeholder(tf.float32, [None, 784])
        self.x2 = tf.placeholder(tf.float32, [None, 784])

        with tf.variable_scope("Siamese") as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        self.loss = self.loss_with_spring()

    def network(self, x):
        
        x = tf.reshape(x,shape=[-1,28,28,1])
        activated_conv1 = self.conv_layer('conv_1',x,6)
        maxpool1 = self.maxpool_layer('maxp_1',activated_conv1)
        
        flattened_conv = tf.layers.flatten(maxpool1)
        activated_fc1 = self.fc_layer( "fc1",flattened_conv, 1024)
        activated_fc2 = self.fc_layer("fc2",activated_fc1, 1024)
        activated_fc3 = self.fc_layer("fc3",activated_fc2, 2)
        
        return activated_fc3
        
    def conv_layer(self,name,inputs,cur_channel):
        prev_channel = inputs.get_shape()[3]
        init = tf.variance_scaling_initializer(scale=2.0)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[5,5,prev_channel,cur_channel],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_channel],initializer = init)
        conv = tf.nn.conv2d(inputs,w,strides=[1,1,1,1],padding = "SAME")
        activation = tf.nn.relu(conv+b)
        return activation  
      
    def maxpool_layer(self,name,inputs):
        return tf.nn.max_pool(inputs,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")  
    
    def fc_layer(self,name,inputs,cur_layer):
        prev_layer = inputs.get_shape()[-1]
        init = tf.truncated_normal_initializer(stddev=0.01)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[prev_layer,cur_layer],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_layer],initializer=init)
        activation = tf.nn.relu(tf.matmul(inputs,w)+b)
        return activation


    def loss_with_spring(self):
        margin = 5.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(margin, name="C")
        # yi*||CNN(p1i)-CNN(p2i)||^2 + (1-yi)*max(0, C-||CNN(p1i)-CNN(p2i)||^2)
        pos = tf.multiply(labels_t, eucd2, name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.subtract(0.0,eucd2), name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C,eucd2)), name="Nyi_x_C-eucd_xx_2")
        neg = tf.multiply(labels_f, tf.pow(tf.maximum(tf.subtract(C, eucd), 0), 2), name="Nyi_x_C-eucd_xx_2")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

In [0]:


# prepare data and tf.session
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
tf.reset_default_graph()
sess = tf.InteractiveSession()


# setup siamese network
siamese = Siamese();
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(siamese.loss)

tf.initialize_all_variables().run()


for step in range(100):
    batch_x1, batch_y1 = mnist.train.next_batch(128)
    batch_x2, batch_y2 = mnist.train.next_batch(128)
    batch_y = (batch_y1 == batch_y2).astype('float')
    
    _, loss_v = sess.run([train_step,siamese.loss], feed_dict={
                        siamese.x1: batch_x1,
                        siamese.x2: batch_x2,
                        siamese.y_: batch_y})

    if np.isnan(loss_v):
        print('Model diverged with loss = NaN')
        quit()

    if step % 10 == 0:
        print ('step %d: loss %.3f' % (step, loss_v))

    if step % 1000 == 0 and step > 0:
        #saver.save(sess, './model')
        embed = siamese.o1.eval({siamese.x1: mnist.test.images})
        embed.tofile('embed.txt')

# visualize result
x_test = mnist.test.images.reshape([-1, 28, 28])
y_test = mnist.test.labels
#visualize.visualize(embed, x_test, y_test)

sess.close()
    

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations 

NameError: ignored